In [2]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
import numpy as np

In [6]:
news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1')
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1')
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1')

news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower()
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw['company_name'] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

# 맵핑한 데이터 저장하기 위한 csv파일 생성
project_result = pd.DataFrame(columns=['news_id', 'company_word','pk'])

In [7]:
news_raw

,news_id,title,summary,tokenize,company_name,keyword,keyword2
0,345585,Nestle Outshines Rivals With Revenue Growth Le...,nestle sa sailed past ailing consumer goo...,"[nestle, sa, sailed, past, ailing, consumer, g...",0,0,0
1,345586,Total Makes Surprise Profit as Trading Gains O...,total se made a surprise profit after ve...,"[total, se, made, a, surprise, profit, after, ...",0,0,0
2,345590,Airbus Follows Boeing in Paring Output to Weat...,airbus se cut back wide body jet producti...,"[airbus, se, cut, back, wide, body, jet, produ...",0,0,0
3,345591,Virus Uptick Imperils South Europe's Nascent T...,nadine scheiner s efforts to travel from ...,"[nadine, scheiner, s, efforts, to, travel, fro...",0,0,0
4,345595,PG&E Fire Insurance Costs Skyrocket After Bank...,pg&e corp is finding it very costly to b...,"[pg&e, corp, is, finding, it, very, costly, to...",0,0,0
...,...,...,...,...,...,...,...
995,443773,Localiza to Form $9 Billion Car-Rental Giant W...,localiza rent a car sa and unidas two of...,"[localiza, rent, a, car, sa, and, unidas, two,...",0,0,0
996,443778,Mendoza Province Is First to Restructure After...,argentina s mendoza province has reached ...,"[argentina, s, mendoza, province, has, reached...",0,0,0
997,443779,Manufacturing Keeps German Economy on a Recove...,german factories kept europe s biggest ec...,"[german, factories, kept, europe, s, biggest, ...",0,0,0
998,443780,ECB Must Limit Emergency Powers to Temporary C...,the european central bank risks legal tro...,"[the, european, central, bank, risks, legal, t...",0,0,0


In [ ]:
## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )
# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                project_result.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        project_result.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            project_result.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

In [8]:
project_result = pd.merge(project_result, nasdaq_stock, how='left', left_on='pk', right_on='pk')

In [9]:
project_result

,news_id,company_word,pk,symbol,name


## 키워드 추출 ( Keybert 사용 )=> 추출 완료 ( keyword.csv 에 저장 )

In [ ]:
# Keybert로 키워드 추출 ( 키워드 1-2 자리까지 )
summaries = news_raw['summary']

for sum_num, summary in enumerate(summaries):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(summary)
    news_raw['keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1))
    news_raw['keyword2'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 2))
#news_raw.to_csv('C:/projectnasdaq/project2/keyword.csv')

## 종목 정보 수집 ( yfinance 패키지 사용 )

In [ ]:
# 종목코드 리스트
stocks = nasdaq_stock['symbol']
# 종목코드에 대한 정보 수집용 데이터프레임 생성
stock_info = pd.DataFrame(columns=['symbol', 'info'])
i = 0

for stock_num, stock in enumerate(stocks):
    tickers = yf.Ticker(stock)
    ticker = tickers.info
    i = i + 1
    #print(stock_num,stock, "===>>",ticker)
    stock_info.loc[i] = [stock, ticker]
    time.sleep(random.uniform(3, 4))

stock_info.to_csv('C:/projectnasdaq/project2/stock_info2.csv')

## 추출된 회사 이름이랑 ( 전처리 한 ) 키워드 연결 ( company_and_keyword.csv 파일에 저장 )

In [ ]:
company_mapping = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')
keyword_mapping = pd.read_csv('C:/projectnasdaq/project2/keyword.csv')

keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(
    lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(
    lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
keyword_mapping['keyword'] = keyword_mapping['keyword'].str.split(" ")
#keyword_mapping['keyword'].replace(' ',',', regex=True, inplace=True)

keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].str.split("  ")
# 공백으로 토큰화되거나 비어있는 내용으로 토큰화 되어있으면 지우기
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: [i for i in x if i != "" and i != " "])

company_mapping[['keyword', 'keyword2']] = 0

keyword_list1 = keyword_mapping['keyword']
keyword_list2 = keyword_mapping['keyword2']

keyword_id = keyword_mapping['news_id']
company_id = company_mapping['news_id']

In [ ]:
for i, j in enumerate(company_id):
    for k, l in enumerate(keyword_id):
        if j == l:
            company_mapping['keyword'][i] = keyword_mapping['keyword'][k]
            company_mapping['keyword2'][i] = keyword_mapping['keyword2'][k]

company_mapping.to_csv('C:/projectnasdaq/project2/company_and_keyword.csv')


In [21]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset.csv')
yahoo_dataset_symbol = yahoo_dataset['symbol']
nasdaq_stock_symbol = nasdaq_stock['symbol']

for i, j in enumerate(yahoo_dataset_symbol):
    for k, l in enumerate(nasdaq_stock_symbol):
        if j == l:
            yahoo_dataset['new_symbol'][i] = nasdaq_stock['symbol'][k]
            break


NameError: name 'nasdaq_stock' is not defined

In [19]:
yahoo_dataset.to_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')
yahoo_dataset.drop(columns=["quoteType", "currency", 'regularMarketPrice', 'regularMarketChange', 'regularMarketChangePercent',
             'regularMarketVolume', 'averageDailyVolume3Month', 'marketCap', 'trailingPE', 'fiftyTwoWeekLow',
             'fiftyTwoWeekHigh', 'regularMarketOpen', 'priceHint', 'underlyingSymbol'], inplace=True)

NameError: name 'yahoo_dataset' is not defined

In [3]:
# company단어들 하나씩만 list에 담기 -> 단어 이름이 같은 종목명들의 키워드 모으기 위해서

company_and_keyword = pd.read_csv('C:/projectnasdaq/project2/company_and_keyword.csv')
company_and_keyword.drop(columns=["Unnamed: 0", 'Unnamed: 0.1'], inplace=True)
company_word = company_and_keyword['company_word']

company_word_same_list = []
for company in company_word:
    if company not in company_word_same_list:
        company_word_same_list.append(company)

In [35]:
company_word_sames = pd.DataFrame(company_word_same_list, columns=['company_word_same'])
company_word_same = company_word_sames['company_word_same']
company_word_sames

,company_word_same
0,coca cola
1,unilever
2,diageo
3,anheuser busch
4,starbucks
5,royal dutch
6,equinor
7,bp
8,eni
9,natural gas


In [49]:
# 키워드랑 company_name이랑 데이터프레임에서 맵핑 시키기위한 새로운 dataframe 생성
company__name = pd.DataFrame(columns=['company', 'keyword','keyword2'])
company__name

,company,keyword,keyword2


In [50]:
# company_name에 대한 키워드들 리스트로 모아서 column에 넣기
for i, j in enumerate(company_word_same):
    company__name.loc[i] = [j, company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].tolist(), company_and_keyword[company_and_keyword['company_word'] == j]['keyword2'].tolist()]

company__name

,company,keyword,keyword2
0,coca cola,"[['nestle', 'starbucks', 'brands', 'bottled', ...","[['nestle sa', 'nestle', 'underperforming bran..."
1,unilever,"[['nestle', 'starbucks', 'brands', 'bottled', ...","[['nestle sa', 'nestle', 'underperforming bran..."
2,diageo,"[['nestle', 'starbucks', 'brands', 'bottled', ...","[['nestle sa', 'nestle', 'underperforming bran..."
3,anheuser busch,"[['nestle', 'starbucks', 'brands', 'bottled', ...","[['nestle sa', 'nestle', 'underperforming bran..."
4,starbucks,"[['nestle', 'starbucks', 'brands', 'bottled', ...","[['nestle sa', 'nestle', 'underperforming bran..."
5,royal dutch,"[['profit', 'earnings', 'markets', 'market', '...","[['market crisis', 'outperformance trading', '..."
6,equinor,"[['profit', 'earnings', 'markets', 'market', '...","[['market crisis', 'outperformance trading', '..."
7,bp,"[['profit', 'earnings', 'markets', 'market', '...","[['market crisis', 'outperformance trading', '..."
8,eni,"[['profit', 'earnings', 'markets', 'market', '...","[['market crisis', 'outperformance trading', '..."
9,natural gas,"[['profit', 'earnings', 'markets', 'market', '...","[['market crisis', 'outperformance trading', '..."


In [46]:
company__name.to_csv('C:/projectnasdaq/project2/test.csv')